# huggingface demo repo with `jinaai/jina-embeddings-v2-base-zh`

demo for pipeline from huggingface

In [10]:
import os
import torch
import pandas as pd
import evaluate
import numpy as np
import joblib
from datasets import Dataset, concatenate_datasets, load_dataset

from sklearn.preprocessing import LabelEncoder
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, DataCollatorWithPadding,
    pipeline, AutoModel, infer_device, TextClassificationPipeline
)

In [11]:
MODEL = 'jinaai/jina-embeddings-v2-base-zh'
DEVICE = infer_device()
DEVICE


'cuda'

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL, 
    trust_remote_code=True
    )
model

Some weights of JinaBertForSequenceClassification were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-zh and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


JinaBertForSequenceClassification(
  (bert): JinaBertModel(
    (embeddings): JinaBertEmbeddings(
      (word_embeddings): Embedding(61056, 768, padding_idx=0)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): JinaBertEncoder(
      (layer): ModuleList(
        (0-11): 12 x JinaBertLayer(
          (attention): JinaBertAttention(
            (self): JinaBertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): JinaBertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

In [15]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [17]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

sentences = ['How is the weather today?', '今天天气怎么样?']

tokenizer = AutoTokenizer.from_pretrained('jinaai/jina-embeddings-v2-base-zh')
model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-zh', trust_remote_code=True, dtype=torch.bfloat16)

encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

with torch.no_grad():
    model_output = model(**encoded_input)

embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
embeddings = F.normalize(embeddings, p=2, dim=1)


In [20]:
embeddings.shape

torch.Size([2, 768])

In [29]:
model.config.model_max_length

8192

In [30]:
model.config.max_position_embeddings 

8192

In [ ]:
# Methods and `google-bert/bert-base-chinese`
dataset = load_dataset(
    'parquet', 
    data_files={
        'train': 'data/train_zh.parquet',
        'test': 'data/test_zh.parquet'
    })
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text', '__index_level_0__'],
        num_rows: 4917
    })
    test: Dataset({
        features: ['label', 'text', '__index_level_0__'],
        num_rows: 258
    })
})

In [ ]:
pipeline = TextClassificationPipeline(
    task='text-classification',
    
)

TypeError: Pipeline.__init__() missing 1 required positional argument: 'model'